In [1]:
# -*- coding: utf-8 -*-
"""
@author: ChewingGumKing_OJF
"""

#loads necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from datetime import datetime
from datetime import date
import openpyxl
from selenium.webdriver.common.action_chains import ActionChains
import os
import warnings
import json
import pandas as pd
import pyperclip as pc
from bs4 import BeautifulSoup as bs
import re
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

warnings.filterwarnings("ignore")


In [2]:
#driver=webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe')
from selenium import webdriver 



chrome_options = Options()

chrome_options.add_argument("--disable-extensions")
# linux only
# chrome_options.add_argument("--headless")
#chrome_options.headless = True 

driver = webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe',options=chrome_options)

In [3]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service

# s = Service(r'C:\Users\840 g3\Desktop\chromedriver.exe')
# driver = webdriver.Chrome(service=s)

In [4]:
url = "https://theilp.org.uk/events/"

In [5]:
driver.get(url)

In [6]:
soup = bs(driver.page_source,'lxml')

In [7]:
div = "ect-events-list-content" #class

In [8]:
section = soup.find('div',{'id':div})

In [9]:
each = section.find_all('div',{'class':'ect-list-post style-1 ect-simple-event future'})

In [10]:
theilp=[]
sidebar = "epta-sidebar-area" #div class
venue = "tribe-events-meta-group tribe-events-meta-group-venue" #div class
cost = "tribe-events-event-cost"#dd class
event_url ='tribe-events-event-url' #dd clas
email = "tribe-organizer-email" #dd class
for i in each:

    try:
        ven = i.find('div',{'class':'ect-list-venue default-venue'}).text.replace('\n\n',' ').replace('\n',' ').split('+')[0].strip()
    except:
        ven='Hosted Online'
        
    
    try:
        cost = i.find('div',{'class':'ect-rate-area'}).text.replace('\t','').replace('\n','')
    except:
        cost=''
    if cost=='':
        ticketlist=''
    elif 'Free' in cost:
        ty='Free'
        am=''
        cu=''
        ticketlist={'type': ty,
                     'price':am,
                     'currency': cu}
    elif '-' in cost:
        price=cost.split('-')[0][1:].replace(',','')
        ty='Paid'
        am=price
        cu=cost[0]
        ticketlist={'type': ty,
                     'price':am,
                     'currency': cu}
        
    else:
        ticketlist=''
    
    if ticketlist=='':
        ticket_list=ticketlist
    else:
        ticket_list=json.dumps([ticketlist])
    
    topic = i.h2.text.replace('\t','').replace('\n','')
    link = i.a['href']
#    data.append([topic,ven,cost,link])


    driver.get(link)
    time.sleep(5)
    soupn = bs(driver.page_source,'lxml')

    sidebar_ = soupn.find('div',{'class':sidebar})
    if str(sidebar_)!='None':
      #  print('aaa')
        try:
            address = sidebar_.find('div',{'class':"tribe-events-meta-group tribe-events-meta-group-venue"}).text.replace('Venue','').replace('\n\n\n','').replace('\n',' ').split('+')[0].strip()

        except:
            address = 'NA'
        try:
            web = sidebar_.find('dd',{'class':event_url}).text.strip()
            if len(url)<=5:
                web=link
        except:
            web = link
        try:
            email = sidebar_.find('dd',{'class':email}).text.strip()
            if len(email)<=5:
                email='NA'
        except:
            email = 'NA'
            
        #***********
        sidebar_ = soupn.find('div',{'class':'epta-sidebar-area'})
        sure=sidebar_.find('div',{'class':'tribe-events-meta-group tribe-events-meta-group-details'})
        title=soupn.find('div',{'class':'epta-title-date'}).h2.text.replace('\n','').replace('\t','')
        if 'Start' and 'End' in str(sure):
          #  print('schs')
            sak=sure.find_all('dd')[:2]
            siuu=[a.text.replace('\n','').replace('\t','').split('@') for a in sak]
            start,end=siuu
            yr=str(date.today().year)
            start=[[a.strip() for a in start][0]+' '+yr,[a.strip() for a in start][1].replace(' ','').replace('pm','PM').replace('am','AM')]
            end=[[a.strip() for a in end][0]+' '+yr,[a.strip() for a in end][1].replace(' ','').replace('pm','PM').replace('am','AM')]
            dat=[start[0],end[0]]
#            dat=[a.replace('st','').replace('nd','').replace('rd','').replace('th','').replace('Augu','August')]
            tim=[start[1],end[1],'']

            #siuu=[a.text.replace('\n','') for a in sak]
        elif 'Date' in str(sure):
           # print('uiuh')
            sak=sure.find_all('dd')[:2]
            siuu=[a.text.replace('\n','').replace('\t','').strip() for a in sak]
            yr=str(date.today().year)
            dat,tim=siuu
            if not re.search('\d{4}',dat):
                dat=[dat+' '+yr,dat+' '+yr]
#                dat=[a.replace('st','').replace('nd','').replace('rd','').replace('th','').replace('Augu','August')]
                tim=tim.replace(' ','').replace('pm','PM').replace('am','AM').split('-')+['']# for a in tim]
        else:
            title=topic
            dat=''
            tim=''
    else:
       # print('bbb')
        noside=soupn.find('div',{'class':'tribe-events-single tribe-blocks-editor'})
        title=noside.find('h1',{'class':'tribe-events-single-event-title'}).text
        timer=noside.find('h2',{'class':'tribe-events-schedule__datetime'}).get_text(separator='').replace('\t','').replace('\n','').replace('–','-').split('-')
        shi=[a.split('@') for a in timer]
        start,end=shi
        yr=str(date.today().year)
        start=[[a.strip() for a in start][0]+' '+yr,[a.strip() for a in start][1].replace(' ','').replace('pm','PM').replace('am','AM')]
        end=[[a.strip() for a in end][0]+' '+yr,[a.strip() for a in end][1].replace(' ','').replace('pm','PM').replace('am','AM')]
        if not re.search('\d{4}',start[0]) and re.search('\d{4}',start[1]):
            dat=[start[0],end[0]]
#            dat=[a.replace('st','').replace('nd','').replace('rd','').replace('th','').replace('Augu','August')]
            tim=[start[1],end[1],'']
        else:
            dat=''
            tim=''
        address=''
        web=link
        email=''
        
    #**************************
    if dat=='':
        start_date=end_date=''
    else:
        dat=[a.replace('st','').replace('nd','').replace('rd','').replace('th','').replace('Augu','August') for a in dat]
        spl_dt_obj = [datetime.strptime(v, '%d %B %Y') for v in dat]
        date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
        start_date=date_[0]
        end_date=date_[1]
        
    if tim=='':
        time_st=''
    else:
        time_st={'type': 'general',
                 'start_time':tim[0],
                 'end_time': tim[1],
                 'timezone': tim[2],
                 'days': 'all'}
    if email=='' or email=='NA' or '@' not in email or '.net' not in email or '.org' not in email or '.com' not in email:
        mail='info@theilp.org.uk'
        mail_=json.dumps([mail])
    else:
        mail_=json.dumps([email])
    if time_st=='':
        time_=''
    else:
        time_=json.dumps([time_st])
    event_web=web
    
     ###############COUNTYR,GPU#################
    def countr(locale):
        try:
            google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
            time.sleep(randint(0,3))
            driver.get(google_url_for_location)
            time.sleep(4)
            try:
                google_map_url=driver.find_element_by_id('lu_map').click()
            except:
                try:
                    google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
                except:
                    google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
            time.sleep(1)
            try:
                country=driver.find_element_by_class_name('x3AX1-LfntMc-header-title-VdSJob').text
            except:
                country=driver.find_element_by_class_name('bwoZTb').text
#                 print(country)
            return(country)
        except Exception as e:
            print(locale, "; didn't go through")
            print(e)
            return("")

    ####################################
    def get_google_map_url(location):
        try:
            google_url_for_location="https://www.google.com/search?q="+location+"&oq="+location+"&num=1"
            time.sleep(randint(0,3))
            driver.get(google_url_for_location)
            time.sleep(4)
            try:
                google_map_url=driver.find_element_by_id('lu_map').click()
            except:
                try:
                    google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
                except:
                    google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
            time.sleep(1)
            google_map_url=driver.current_url
#                 print(google_map_url)
            return(google_map_url)
        except Exception as e:
            print(location, "; didn't go through")
            print(e)
            return("")
            

    
    if 'Online' in ven:
        on_off='1'
        city=''
        country=''
        venue=''
        googlePlaceUrl=''
    else:
        on_off='0'
        city=ven
        venue=ven
        country=countr(city)
        googlePlaceUrl=get_google_map_url(venue)
        
        
    org_pro=''
    org_name='Institution of Lighting Professionals'
    org_web='https://theilp.org.uk/'
    
    event_info=''    
    logo=''
    sponsors=''
    agendalist=''
    type_=''
    category=''
    Speakerlist=''
    
    theilp.append([link,title,start_date,end_date,time_,event_info,ticket_list,
                     org_pro,org_name,org_web,logo,sponsors,agendalist,
                     type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
                     Speakerlist,on_off])
print('Done')

Watershed (daytime) & Bristol Hotel (evening of 21.06.22), Multiple locations Bristol, England BS1 5TX United Kingdom  didn't go through
Message: no such element: Unable to locate element: {"method":"css selector","selector":".Lx2b0d"}
  (Session info: chrome=100.0.4896.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x01189943+2595139]
	Ordinal0 [0x0111C9F1+2148849]
	Ordinal0 [0x01014528+1066280]
	Ordinal0 [0x01040FD4+1249236]
	Ordinal0 [0x010411CB+1249739]
	Ordinal0 [0x0106D812+1431570]
	Ordinal0 [0x0105BA34+1358388]
	Ordinal0 [0x0106BAF2+1424114]
	Ordinal0 [0x0105B806+1357830]
	Ordinal0 [0x01036086+1204358]
	Ordinal0 [0x01036F96+1208214]
	GetHandleVerifier [0x0132B232+1658114]
	GetHandleVerifier [0x013E312C+2411516]
	GetHandleVerifier [0x0121F261+560433]
	GetHandleVerifier [0x0121E366+556598]
	Ordinal0 [0x0112286B+2173035]
	Ordinal0 [0x011275F8+2192888]
	Ordinal0 [0x011276E5+2193125]
	Ordinal0 [0x011311FC+2232828]
	BaseThreadInitThunk [0x767EFA29+25]
	RtlGetAppContainerNamedObjectPath [0x76F

In [19]:
len(theilp)

13

In [20]:
theilp

[['https://theilp.org.uk/event/hi-lights-from-the-ilp-7/',
  'Hi Lights from the ILP',
  '2022-04-25',
  '2022-04-25',
  '[{"type": "general", "start_time": "1:00PM", "end_time": "3:00PM", "timezone": "", "days": "all"}]',
  '',
  '[{"type": "Free", "price": "", "currency": ""}]',
  '',
  'Institution of Lighting Professionals',
  'https://theilp.org.uk/',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'https://www.eventbrite.co.uk/e/hi-lights-from-the-ilp-tickets-256708832027',
  '',
  '["info@theilp.org.uk"]',
  '',
  '1'],
 ['https://theilp.org.uk/event/how-to-be-brilliant-at-co-design/',
  'How To Be Brilliant at Co-Design',
  '2022-04-26',
  '2022-04-26',
  '[{"type": "general", "start_time": "6:00PM", "end_time": "9:00PM", "timezone": "", "days": "all"}]',
  '',
  '[{"type": "Free", "price": "", "currency": ""}]',
  '',
  'Institution of Lighting Professionals',
  'https://theilp.org.uk/',
  '',
  '',
  '',
  '',
  '',
  'Arup, 10 George Street Edinburgh, England EH2 2PF Unite

In [21]:
theilp_dict=theilp


theilp_df= pd.DataFrame(columns=['scrappedUrl','eventname','startdate','enddate','timing','eventinfo','ticketlist','orgProfile','orgName','orgWeb','logo','sponsor','agendalist','type','category','city','country','venue','event_website','googlePlaceUrl','ContactMail','Speakerlist','online_event'],data=theilp_dict)
theilp_df.to_csv("theilp.tsv", sep = '\t',index=False)
print('Dusted')

Dusted


In [14]:
# start_date,end_date,time_st,address,web,email

In [15]:
# driver.current_url

In [16]:
# soupn = bs(driver.page_source,'lxml')
# noside=soupn.find('div',{'class':'tribe-events-single tribe-blocks-editor'})
# title=noside.find('h1',{'class':'tribe-events-single-event-title'}).text
# timer=noside.find('h2',{'class':'tribe-events-schedule__datetime'}).get_text(separator='').replace('\t','').replace('\n','').replace('–','-').split('-')
# shi=[a.split('@') for a in timer]
# start,end=shi
# yr=str(date.today().year)
# start=[[a.strip() for a in start][0]+' '+yr,[a.strip() for a in start][1].replace(' ','').replace('pm','PM').replace('am','AM')]
# end=[[a.strip() for a in end][0]+' '+yr,[a.strip() for a in end][1].replace(' ','').replace('pm','PM').replace('am','AM')]
# if not re.search('\d{4}',start[0]) and re.search('\d{4}',start[1]):
#     dat=[start[0],end[0]]
#     tim=[start[1],end[1],'']

In [17]:
# dat,tim

In [18]:
# soupn = bs(driver.page_source,'lxml')
# sidebar_ = soupn.find('div',{'class':'epta-sidebar-area'})
# sure=sidebar_.find('div',{'class':'tribe-events-meta-group tribe-events-meta-group-details'})
# title=soupn.find('div',{'class':'epta-title-date'}).h2.text.replace('\n','').replace('\t','')
# if 'Start' and 'End' in str(sure):
#     sak=sure.find_all('dd')[:2]
#     siuu=[a.text.replace('\n','').replace('\t','').split('@') for a in sak]
#     start,end=siuu
#     yr=str(date.today().year)
#     start=[[a.strip() for a in start][0]+' '+yr,[a.strip() for a in start][1].replace(' ','').replace('pm','PM').replace('am','AM')]
#     end=[[a.strip() for a in end][0]+' '+yr,[a.strip() for a in end][1].replace(' ','').replace('pm','PM').replace('am','AM')]
#     dat=[start[0],end[0]]
#     tim=[start[1],end[1],'']
    
#     #siuu=[a.text.replace('\n','') for a in sak]
# elif 'Date' in str(sure):
#     sak=sure.find_all('dd')[:2]
#     siuu=[a.text.replace('\n','').replace('\t','').strip() for a in sak]
#     yr=str(date.today().year)
#     dat,tim=siuu
#     if not re.search('\d{4}',dat):
#         dat=[dat+' '+yr,dat+' '+yr]
#         tim=tim.replace(' ','').replace('pm','PM').replace('am','AM').split('-')+['']# for a in tim]
# else:
#     title=''
#     dat=''
#     tim=''